In [ ]:
import warnings, os
import datetime as dt
import itertools as it
from numpy import nan, inf
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

# Load

In [ ]:
Raw = pd.read_csv('Storage/Metatrader_WINN.csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin(SYMBOLS) & Raw['tf'].isin(TFRAMES)] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=['WIN$N'], TFRAMES=['M5','M15','H1','H3','D1','W1'])
Query.head()

# Data

In [ ]:
def FORMAT_SOURCE(Query):
    Src = pd.DataFrame()
    Src['A']        = Query['a']
    Src['Z']        = Query['z']
    Src['Symbol']   = Query['symbol']
    Src['TF']       = Query['tf']
    
    Src['Datetime'] = Query['datetime']
    Src['Date']     = Query['datetime'].dt.date
    Src['Time']     = Query['datetime'].dt.time
    
    Src['Ticks']    = Query['tick_volume']
    Src['Volume']   = Query['real_volume']
    Src['Price']    = Query['close']
    
    Src['Open']     = Query['open']
    Src['High']     = Query['high']
    Src['Low']      = Query['low']
    Src['Close']    = Query['close']

    Src['Change']   = Src['Close'] - Src['Open']
    Src['HL']       = Src['High']  - Src['Low']
    return Src 

Src = FORMAT_SOURCE(Query)
Src.head()

In [ ]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc['Chg Abs']         = Calc['Change'].abs() 
    Calc['Chg Abs Avg']     = Calc.groupby('TF', sort=0)['Chg Abs'] .expanding().mean()     .reset_index(drop=1)
    Calc['HL Avg']          = Calc.groupby('TF', sort=0)['HL']      .expanding().mean()     .reset_index(drop=1)

    Calc['Chg Abs > Avg']   = Calc['Chg Abs'] > Calc['Chg Abs Avg']
    Calc['HL > Avg']        = Calc['HL']      > Calc['HL Avg']
    return Calc

Calc = CALCULATIONS(Src)
Calc.head()

# Stats

In [10]:
Calc.groupby('TF', sort=0)[['HL > Avg','Chg Abs > Avg']].mean().mul(100).round(0).astype(int)

,HL > Avg,Chg Abs > Avg
TF,,
M5,33,33
M15,33,33
H1,33,33
H3,35,33
D1,34,38
W1,40,44


In [9]:
Calc.groupby('TF', sort=0)[['HL','Chg Abs']].apply(lambda S: (S > S.mean()).mean()).mul(100).round(0).astype(int)

,HL,Chg Abs
TF,,
M5,38,35
M15,39,36
H1,40,37
H3,43,38
D1,40,40
W1,43,42


# Snippets

In [ ]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc['Chg Abs']         = Calc['Change'].abs() 
    Calc['Chg Abs Avg']     = Calc.groupby('TF', sort=0)['Chg Abs'] .expanding().mean()     #.reset_index(drop=1)
    Calc['HL Avg']          = Calc.groupby('TF', sort=0)['HL']      .expanding().mean()     #.reset_index(drop=1)

    Calc['Chg Abs > Avg']   = Calc['Chg Abs'] > Calc['Chg Abs Avg']
    Calc['HL > Avg']        = Calc['HL']      > Calc['HL Avg']
    return Calc

# Each value is compared with full timeline average
Calc.groupby('TF', sort=0)[['HL','Chg Abs']].apply(lambda S: (S > S.mean()).mean()) .mul(100).round(0).astype(int)

# Each value is compared with only past window average
Calc.groupby('TF', sort=0)[['HL > Avg','Chg Abs > Avg']].mean()                     .mul(100).round(0).astype(int)